<center> <img src="https://oci02.img.iteso.mx/Identidades-De-Instancia/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg" align="center" width="650" height="400"/> </center>

<center><font color=#555555><font size=6> MAF1731 - Microstructure and Trading Systems </font> <br> <br>
    
<center><font color=#555555><font size=5> B.Eng Financial Engineering </font> <br> <br> <br> <br>

<center><font color=#555555><font size=6> I.F. Gerardo González </font> <br> <br>
    
<center><font color=#555555><font size=6> I.F. Tania Bell Nicholls</font> <br> <br>

<center> <font color= #555555> <font size = 4> April 2023 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/Tania-Bell/MyST_Lab4'>Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Lab 4 </font> <br> <br> <font color= #555555> <font size = 5>Market's Microstructure </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

<font color= #555555> <font size = 4> Abstract </font>

<font color= #6B6B6B> <font size = 3>  Throughout this laboratory, we will use the information from the Cryptocurrency's Book of Orders from various international exchanges, with the primary objective of visualizing and modeling the market's microstructure through it's characteristics and behaviors like Matchine Engine, Levels, Ask-Bid Volumes, etc.    </font>

<br>

# <font color= #6B6B6B> 0. Install/Load Packages and Depedencies </font>

<hr style="border:0.02in solid gray"> </hr>

## <font color= #6B6B6B> 0.2 Python Packages </font> <font color= #555555>

In order to run this notebook, it is necessary to have installed and/or have the _requirements.txt_ file with the following: 

- pandas
- numpy
- jupyter
- ccxt
- ast
- time
- asyncio

## <font color= #6B6B6B> 0.3 Files Dependencies </font> <font color= #555555>

The following are the file dependencies that are needed to run this notebook:

- files/dat.csv : DataFrame containing book of orders information

## <font color= #6E6E6E> 0.4 Install Packages</font>

In [1]:
%%capture

# Install all the pip packages in the requirements.txt
import sys
!{sys.executable} -m pip install -r requirements.txt

In [4]:
# importing all libraries
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta
import numpy as np
import pandas as pd
import ccxt
import functions as fn
import data as dat
import main as mn
import visualizations as vn

In [7]:
import pandas as pd

data = pd.read_csv('files/dat.csv')
data_volumes = pd.read_csv('files/data_volumes.csv')

# <font color= #6B6B6B> <font size = 5> 1. Introduction </font>

This laboratory focuses on using the information in the cryptocurrencies' Book of Orders to visualize and model the market's microstructure. In order to learn about a market's microstructure it is important to have all the information available, for that reason the market chosen for this project was the cryptocurrecy.

This project starts with obtaining the most accurate and current information of three different exchange markets and three different trading pairs using the ccxt library. The exchange markets chosen were Binance, Bitfinex and Kraken. The trading pairs chosen were BTC/USDT, ETH/USDT and ADA/USDT. Then the information was organized and summarized taking into account the levels and total bid size and ask size.

For the visualization, graphs of the mid price were generated with the plotly library in order to compare the different time series.

To complete the objetives, the last part of the lab consisted of modeling the microstructure of the market. This was achieved through estimating the Effective Spread based on the Roll model (1984).

# <font color= #6B6B6B> <font size = 5> 2. Objectives </font>

- Obtain, clean and analyze the information obtained from the Book of Orders
- Visualize the mid price time series
- Model the market's microstructure using the effective spread

# <font color= #6B6B6B> <font size = 5> 3. Data Description </font>

<hr style="border:0.02in solid gray"> </hr>

Exchange markets:
- Binance
- Bitfinex
- Kraken

Trading pairs:
- BTC/USDT
- ETH/USDT
- ADA/USDT

Datetime : timestamp in BO

Ask_size : volume of asks

Ask_price : price of asks

Bid_size : volume of bids

Bid_price : price of bids

Spread : difference between ask and bid

Mid_price : average Top of the Book

VWAP : volume-weighted average price

Close_price : closing price

Levels : trading levels

In [6]:
data # it is a dictionary but for visualization terms in the notebook it was downloaded as csv

,Unnamed: 0,exchange,datetime,orderbook
0,0,binance,2023-04-20T03:20:19.463Z,"{'ask_size': [25.8783, 0.0014, 0.00045, 0.0007..."
1,1,bitfinex,2023-04-20T03:20:41.657Z,"{'ask_size': [0.190762, 0.15, 0.11160896, 0.45..."
2,2,kraken,2023-04-20T03:20:22.197Z,"{'ask_size': [0.07, 0.002, 0.007, 0.05, 0.003,..."
3,3,binance,2023-04-20T03:20:54.307Z,"{'ask_size': [9.1926, 0.05, 1.7577, 0.0429, 0...."
4,4,bitfinex,2023-04-20T03:21:12.401Z,"{'ask_size': [3.5973, 2.19558735, 0.8027098, 3..."
5,5,kraken,2023-04-20T03:20:52.362Z,"{'ask_size': [0.7, 0.784, 0.7, 9.689, 0.7, 4.3..."
6,6,binance,2023-04-20T03:21:19.536Z,"{'ask_size': [288233.8, 14506.5, 48170.7, 6231..."
7,7,bitfinex,2023-04-20T03:21:40.091Z,"{'ask_size': [1198.5, 4541.19912461, 2396.0, 1..."
8,8,kraken,2023-04-20T03:21:21.221Z,"{'ask_size': [22.202, 5000.0, 3592.97, 1512.01..."


## <font color= #6B6B6B> <font size = 5> 4. Microstructure Visualizations  </font>

In order to visualize the information obtained from the Book of Orders, a DataFrame was created with the type of exchange market, timestamp, levels, volume of asks, volume of bids, total volume, mid price and VWAP. This created the right structure to then create the time series shown in the following graphs.

In [8]:
data_volumes

,Unnamed: 0,exchange,timestamp,levels,ask_volume,bid_volume,total_volume,mid_price,vwap
0,0,binance,2023-04-20T04:54:31.075Z,100,3.286697e+01,2.901284e+01,6.187981e+01,28880.975000,28883.986686
1,1,bitfinex,2023-04-20T04:54:51.541Z,25,1.435826e+01,9.510462e+00,2.386872e+01,28887.500000,28889.026941
2,2,kraken,2023-04-20T04:54:32.261Z,100,4.158800e+01,3.552900e+01,7.711700e+01,28889.200000,28927.085269
3,3,binance,2023-04-20T04:54:57.194Z,100,5.523844e+02,4.689370e+02,1.021321e+03,1953.565000,1953.599328
4,4,bitfinex,2023-04-20T04:55:18.355Z,25,9.400409e+01,1.201790e+02,2.141831e+02,1953.350000,1953.133632
5,5,kraken,2023-04-20T04:54:58.401Z,100,3.134310e+02,4.329910e+02,7.464220e+02,1953.640000,1947.925705
6,6,binance,2023-04-20T04:55:23.721Z,100,2.172865e+06,2.630674e+06,4.803539e+06,0.419150,0.418632
7,7,bitfinex,2023-04-20T04:55:44.946Z,25,2.242967e+05,1.293507e+05,3.536474e+05,0.419335,0.419487
8,8,kraken,2023-04-20T04:55:24.857Z,100,3.196464e+05,2.965490e+05,6.161953e+05,0.419124,0.419603


### # poner graficas aqui

<br>

## <font color= #6B6B6B> <font size = 5> 5. Microstructure Modeling </font>

In order to model the microstructure of the market, the Effective Spread was calculated based on Roll's model (1984). This model provides a simple market microstructure model that allows one to estimate the bid-ask spread from observed transaction prices alone, without information on the underlying bid-ask price quotes and the order. The model developed a simple model to estimate the bid–ask spread using the serial covariance of observable trade prices. The model used in this project is the following:

$$2\sqrt{|COV(\Delta P_t,\Delta P_{t-1}|}$$ 

using the series in $t-5$ for $\Delta P_{t-1}$.

###  # poner dataframe effective spread aqui

# <font color= #6B6B6B> <font size = 5> 6. Conclusions </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

# <font color= #6B6B6B> <font size = 5> 7. References </font>

<hr style="border:0.02in solid gray"> </hr>

Nimalendran, M. (2010). Roll Model. Encyclopedia of Quantitative Finance. https://doi.org/10.1002/9780470061602.eqf18014

Semiparametric Identification of the Bid-Ask Spread in Extended. (2015). Core. https://core.ac.uk/download/pdf/146463242.pdf